### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#############################
#                           #
#  Homework 4 - Pandas      #
#  Student - Matheus Gratz  #
#                           #
#############################

# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# Cast the df types, just in case :)
purchase_data.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

## Player Count

* Display the total number of players


In [3]:
# Set the empty list
total_players = []

# Calculate the total number of players
total_players.append(len(purchase_data['SN'].unique()))

# Set de Data Frame
total_players_df = pd.DataFrame(total_players, columns = ['Total Players'])

# Display the Data Frame
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# Set the empty dictionary
purchasing_analysis_dict = {}

# Number of Unique Items
num_unique_items = len(purchase_data['Item ID'].unique())
purchasing_analysis_dict['Number of Unique Items'] = num_unique_items

# Average Purchase Price
mean_price = purchase_data['Price'].mean(axis=0)
purchasing_analysis_dict['Average Price'] = mean_price

# Total Number of Purchases
total_num_purchases = purchase_data['Item ID'].count()
purchasing_analysis_dict['Total Number of Purchases'] = total_num_purchases

# Total Revenue
total_revenue = purchase_data['Price'].sum(axis=0)
purchasing_analysis_dict['Total Revenue'] = total_revenue

# Set the summary data frame
purchasing_analysis_df = pd.DataFrame(list(purchasing_analysis_dict.values()))
purchasing_analysis_df = purchasing_analysis_df.transpose()
purchasing_analysis_df.columns = purchasing_analysis_dict.keys()

# Format fields
purchasing_analysis_df['Number of Unique Items'] = purchasing_analysis_df['Number of Unique Items'].map("{:.0f}".format)
purchasing_analysis_df['Total Number of Purchases'] = purchasing_analysis_df['Total Number of Purchases'].map("{:.0f}".format)
purchasing_analysis_df['Average Price'] = purchasing_analysis_df['Average Price'].map("${:.2f}".format)
purchasing_analysis_df['Total Revenue'] = purchasing_analysis_df['Total Revenue'].map("${:,.2f}".format)

# Display the summary data frame
purchasing_analysis_df

,Number of Unique Items,Average Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics



* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
# Set a data frame with unique Player Names
unique_players_df = purchase_data.drop_duplicates(subset=['SN', 'Gender'])

# Create a count column for each gender
count_gender = unique_players_df["Gender"].value_counts()

# Set the total
gender_demographics_df = pd.DataFrame(count_gender)
gender_demographics_df.columns = ["Total Count"]

# Calculate the sum
sum_players = gender_demographics_df['Total Count'].sum()

# Generate te final output
gender_demographics_df['Percentage of Players'] = gender_demographics_df['Total Count'] / sum_players * 100

# Format fields
gender_demographics_df['Percentage of Players'] = gender_demographics_df['Percentage of Players'].map("{:.2f}%".format)

# Display the summary data frame
gender_demographics_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
#Generate the calculations
purchase_analysis_gender_df = purchase_data.groupby('Gender').agg(
    total_users = ('SN', 'nunique'),
    total_orders = ('Purchase ID', 'count'),
    avg_price = ('Price', 'mean'),
    total_revenue = ('Price', 'sum')
    )

# Calculate the average per person
purchase_analysis_gender_df['Average Purchase Total per Person'] = purchase_analysis_gender_df['total_revenue'] / purchase_analysis_gender_df['total_users']

# Rename Columns
purchase_analysis_gender_df = purchase_analysis_gender_df.rename(columns={
    'total_users' : 'Total Users',
    'total_orders' : 'Purchase Count',
    'avg_price' : 'Average Purchase Price',
    'total_revenue' : 'Total Purchase Value',
})

# Format fields
purchase_analysis_gender_df['Average Purchase Price'] = purchase_analysis_gender_df['Average Purchase Price'].map("${:,.2f}".format)
purchase_analysis_gender_df['Total Purchase Value'] = purchase_analysis_gender_df['Total Purchase Value'].map("${:,.2f}".format)
purchase_analysis_gender_df['Average Purchase Total per Person'] = purchase_analysis_gender_df['Average Purchase Total per Person'].map("${:,.2f}".format)

# Display the summary data frame
purchase_analysis_gender_df

,Total Users,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Gender,,,,,
Female,81,113,$3.20,$361.94,$4.47
Male,484,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,11,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Generate min and max ages, stablishing min and max bins.
min_age = purchase_data['Age'].min()
max_age = purchase_data['Age'].max()

# Generate bins by list comprehension
bins = [x for x in range(0, int(max_age)+1, int(max_age/9))]

# Create bin labels
labels = [f"from {round(bins[x])} to {round(bins[x+1])}" for x in range(len(bins)-1)]

# Cut the dataframe in bins
purchase_data_groups = purchase_data
purchase_data_groups['Age Group'] = pd.cut(purchase_data_groups['Age'], bins, labels = labels)

# Calculate fields
age_demographics_df = purchase_data.groupby('Age Group').agg(total_users = ('SN', 'nunique'))

# Create sum measure
sum_ages = age_demographics_df['total_users'].sum()

# Calculate percentages
age_demographics_df['Percentage of Players'] = age_demographics_df['total_users'] / sum_ages * 100

# Format fields
age_demographics_df['Percentage of Players'] = age_demographics_df['Percentage of Players'].map("{:.2f}%".format) 

age_demographics_df = age_demographics_df.rename(columns={'total_users' : 'Total Users'})

# Display the summary data frame
age_demographics_df

,Total Users,Percentage of Players
Age Group,,
from 0 to 5,0,0.00%
from 5 to 10,24,4.17%
from 10 to 15,41,7.12%
from 15 to 20,150,26.04%
from 20 to 25,232,40.28%
from 25 to 30,59,10.24%
from 30 to 35,37,6.42%
from 35 to 40,26,4.51%
from 40 to 45,7,1.22%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# Calculate fields
purchase_data_groups = purchase_data.groupby('Age Group').agg(
    total_users   = ('SN', 'nunique'),
    total_orders  = ('Purchase ID', 'count'),
    avg_price     = ('Price', 'mean'),
    total_revenue = ('Price', 'sum')
    )
purchase_data_groups['Average Purchase Total per Person'] = purchase_data_groups['total_revenue'] / purchase_data_groups['total_users']

# Rename Columns
purchase_data_groups = purchase_data_groups.rename(columns={
    'total_users' : 'Total Users',
    'total_orders' : 'Purchase Count',
    'avg_price' : 'Average Purchase Price',
    'total_revenue' : 'Total Purchase Value',
})

# Format fields
purchase_data_groups['Average Purchase Price'] = purchase_data_groups['Average Purchase Price'].map("${:,.2f}".format)
purchase_data_groups['Total Purchase Value'] = purchase_data_groups['Total Purchase Value'].map("${:,.2f}".format)
purchase_data_groups['Average Purchase Total per Person'] = purchase_data_groups['Average Purchase Total per Person'].map("${:,.2f}".format)

# Display the summary data frame
purchase_data_groups

,Total Users,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Group,,,,,
from 0 to 5,0,0,$nan,$0.00,$nan
from 5 to 10,24,32,$3.40,$108.96,$4.54
from 10 to 15,41,54,$2.90,$156.60,$3.82
from 15 to 20,150,200,$3.11,$621.56,$4.14
from 20 to 25,232,325,$3.02,$981.64,$4.23
from 25 to 30,59,77,$2.88,$221.42,$3.75
from 30 to 35,37,52,$2.99,$155.71,$4.21
from 35 to 40,26,33,$3.40,$112.35,$4.32
from 40 to 45,7,7,$3.08,$21.53,$3.08


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Calculate fields
spenders_df = purchase_data.groupby('SN').agg(
    total_orders  = ('Purchase ID', 'count'),
    avg_price     = ('Price', 'mean'),
    total_revenue = ('Price', 'sum')
    )
spenders_df = spenders_df.sort_values('total_orders', ascending=False)

# Rename Columns
spenders_df = spenders_df.rename(columns={
    'total_orders' : 'Purchase Count',
    'avg_price' : 'Average Purchase Price',
    'total_revenue' : 'Total Purchase Value',
})

# Format fields
spenders_df['Average Purchase Price'] = spenders_df['Average Purchase Price'].map("${:,.2f}".format)
spenders_df['Total Purchase Value'] = spenders_df['Total Purchase Value'].map("${:,.2f}".format)

# Display the summary data frame
spenders_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
most_popular_items_df = purchase_data[['Item ID', 'Item Name', 'Price']]

most_popular_items_gp = most_popular_items_df.groupby(['Item ID', 'Item Name']).agg(
    total_orders  = ('Item ID', 'count'),
    avg_price     = ('Price', 'mean'),
    total_revenue = ('Price', 'sum'),
)

most_popular_items_gp = most_popular_items_gp.rename(columns={
    'total_orders' : 'Purchase Count',
    'avg_price' : 'Item Price',
    'total_revenue' : 'Total Purchase Value',
})
most_popular_items_gp.sort_values('Purchase Count', ascending=False, inplace=True)

# Format fields
most_popular_items_gp['Item Price'] = most_popular_items_gp['Item Price'].map("${:,.2f}".format)
most_popular_items_gp['Total Purchase Value'] = most_popular_items_gp['Total Purchase Value'].map("${:,.2f}".format)

# Display the summary data frame
most_popular_items_gp.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
# Unformat fields, replacing the currency symbol and converting to a float
most_popular_items_gp['Total Purchase Value'] = most_popular_items_gp['Total Purchase Value'].replace('[\$,]','',regex=True).astype(float)

# Sort the data
most_popular_items_gp.sort_values('Total Purchase Value', ascending=False, inplace=True)

# Format fields
most_popular_items_gp['Total Purchase Value'] = most_popular_items_gp['Total Purchase Value'].map("${:,.2f}".format)

# Display the summary data frame
most_popular_items_gp.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
